In [ ]:
# Imports
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import pickle
import collections
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_curve
)

In [ ]:
# Use Mixed Precision (save VRAM)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
print("mixed precision enabled.")

In [ ]:
# Load Preprocessed Data --- balanced checked
DATA_PATH = "/kaggle/input/preprocessed-mammo-splits"  
train = np.load(os.path.join(DATA_PATH, "train_data.npz"))
val = np.load(os.path.join(DATA_PATH, "val_data.npz"))
test = np.load(os.path.join(DATA_PATH, "test_data.npz"))

X_train, y_train = train["X"], train["y"]
X_val, y_val = val["X"], val["y"]
X_test, y_test = test["X"], test["y"]


In [ ]:
# Normalize
X_train, X_val, X_test = X_train / 255.0, X_val / 255.0, X_test / 255.0

In [ ]:
# Compute Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print("Class Weights:", class_weight_dict)

In [ ]:
def convert_to_rgb(image, label):
    image_rgb = tf.image.grayscale_to_rgb(image)  
    image_rgb = tf.squeeze(image_rgb) 
    return image_rgb, label

In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Expand dims because TF expects (H, W, 1) from (H, W)
X_train = X_train[..., np.newaxis].astype("float32")
X_val = X_val[..., np.newaxis].astype("float32")
X_test = X_test[..., np.newaxis].astype("float32")

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Shuffle, batch, convert to RGB, prefetch
train_ds = (
    train_ds.shuffle(1024)
    .map(convert_to_rgb, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

val_ds = (
    val_ds.map(convert_to_rgb, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_ds = (
    test_ds.map(convert_to_rgb, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)


In [10]:
def build_model(base_model_fn, name="model"):
    base_model = base_model_fn(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False  

    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)  
    x = GlobalAveragePooling2D()(x) 
    x = Dropout(0.3)(x)  
    x = Dense(128, activation='relu')(x)  
    outputs = Dense(1, activation='sigmoid', dtype='float32')(x)  

    model = Model(inputs, outputs, name=name)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]) 
    return model

In [11]:
# Prepare Callbacks
os.makedirs("/kaggle/working/models", exist_ok=True)
callbacks = lambda name: [
    EarlyStopping(patience=50, restore_best_weights=True),
    ModelCheckpoint(f"/kaggle/working/models/{name}.keras", save_best_only=True)
]

In [ ]:
# Train and Save Models
models = {
    "VGG16": VGG16,
    "ResNet50": ResNet50,
    "DenseNet121": DenseNet121
}

history_dict = {}

for name, fn in models.items():
    print(f"\nTraining {name}...")
    
    # Build model
    model = build_model(fn, name=name)
    
    # Train model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=callbacks(name),
        verbose=2
    )

    # Save model
    model.save(f"{name}_trained_model.h5")
    print(f"Saved model: {name}_trained_model.h5")

    # Save training history
    history_dict[name] = history.history
    with open(f"{name}_history.pkl", "wb") as f:
        pickle.dump(history.history, f)
    print(f"Saved training history: {name}_history.pkl")

print("\n Training completed and all models & histories saved.")



Training VGG16...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100


I0000 00:00:1745307366.998941      90 service.cc:148] XLA service 0x77febc00e190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745307366.999512      90 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745307367.500090      90 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745307377.547918      90 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


555/555 - 79s - 142ms/step - accuracy: 0.5013 - auc: 0.5043 - loss: 0.6957 - val_accuracy: 0.5578 - val_auc: 0.5000 - val_loss: 0.6885
Epoch 2/100
555/555 - 46s - 83ms/step - accuracy: 0.5000 - auc: 0.5003 - loss: 0.6950 - val_accuracy: 0.5578 - val_auc: 0.5004 - val_loss: 0.6869
Epoch 3/100
555/555 - 46s - 82ms/step - accuracy: 0.4980 - auc: 0.4995 - loss: 0.6950 - val_accuracy: 0.5578 - val_auc: 0.5000 - val_loss: 0.6872
Epoch 4/100
555/555 - 46s - 82ms/step - accuracy: 0.4997 - auc: 0.4965 - loss: 0.6951 - val_accuracy: 0.5578 - val_auc: 0.5000 - val_loss: 0.6872
Epoch 5/100
555/555 - 46s - 83ms/step - accuracy: 0.4973 - auc: 0.5022 - loss: 0.6944 - val_accuracy: 0.5578 - val_auc: 0.5000 - val_loss: 0.6865
Epoch 6/100
555/555 - 46s - 83ms/step - accuracy: 0.5028 - auc: 0.5032 - loss: 0.6941 - val_accuracy: 0.5578 - val_auc: 0.5000 - val_loss: 0.6866
Epoch 7/100
555/555 - 46s - 82ms/step - accuracy: 0.5033 - auc: 0.4980 - loss: 0.6949 - val_accuracy: 0.5578 - val_auc: 0.5000 - val_lo